In [1]:
import config

In [2]:
from mysql.connector import errorcode

In [27]:
import mysql.connector as mysql

mydb = mysql.connect(
  host=config.host,
  user=config.admin,
  passwd=config.password,
  database='MLB_Stats'
)

mycursor = mydb.cursor()

#mycursor.execute("CREATE DATABASE MLB_Stats")

In [1]:
import query_helper

In [3]:
query_helper.connect()

In [36]:
tables = {}
tables['games'] = (
    "CREATE TABLE games ("
    "  gameid int(11) NOT NULL,"
    "  game_end_datetime datetime(6) NOT NULL,"
    "  game_date datetime(3) NOT NULL,"
    "  home_id int(3) NOT NULL,"
    "  away_id int(3) NOT NULL,"
    "  status varchar(20) NOT NULL,"
    "  PRIMARY KEY (gameid)"
    ") ENGINE=InnoDB")

In [37]:
tables['game_info'] = (
    "CREATE TABLE game_info ("
    "  gameid int(11) NOT NULL,"
    "  info_id int NOT NULL AUTO_INCREMENT,"
    "  home_team_runs int(3) NOT NULL,"
    "  away_team_runs int(3) NOT NULL,"
    "  venue varchar(50) NOT NULL,"
    "  weather_category varchar(30) NOT NULL,"
    "  temp int(3) NOT NULL,"
    "  wind_mph int(3) NOT NULL,"
    "  wind_direction varchar(40) NOT NULL,"
    "  PRIMARY KEY (info_id),"
    "  FOREIGN KEY (gameid) REFERENCES games(gameid)"
    ") ENGINE=InnoDB")

In [38]:
tables['teams'] = (
    " CREATE TABLE teams("
    " team_id int(11) NOT NULL,"
    " team_name VARCHAR(30) NOT NULL,"
    " league VARCHAR(2) NOT NULL,"
    " PRIMARY KEY (team_id)"
    ") ENGINE=InnoDB")

In [39]:
tables['venues'] = (
    " CREATE TABLE venue("
    " venue_id int(11) NOT NULL AUTO_INCREMENT,"
    " team_id int(11) NOT NULL,"
    " name VARCHAR(30) NOT NULL,"
    " PRIMARY KEY (venue_id),"
    " FOREIGN KEY (team_id) REFERENCES teams(team_id)"
    ") ENGINE=InnoDB")

In [40]:
for table_name in tables:
    table_description = tables[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        mycursor.execute(table_description)
    except mysql.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table games: OK
Creating table game_info: OK
Creating table teams: already exists.
Creating table venues: already exists.


In [41]:
import get_games

In [42]:
df = get_games.df

In [43]:
tuples = [tuple(x) for x in df.values]

In [45]:
tuples

[(446877, '2016-04-03 17:05:00', '2016-04-03', 'Final', 138, 134),
 (446911, '2016-04-03 20:05:00', '2016-04-03', 'Final', 141, 139),
 (446873, '2016-04-04 00:37:00', '2016-04-03', 'Final', 121, 118),
 (446875, '2016-04-04 18:10:00', '2016-04-04', 'Final', 137, 158),
 (446872, '2016-04-04 19:05:00', '2016-04-04', 'Final', 142, 110),
 (446876, '2016-04-04 20:05:00', '2016-04-04', 'Final', 136, 140),
 (446874, '2016-04-04 20:10:00', '2016-04-04', 'Final', 143, 113),
 (446879, '2016-04-04 20:10:00', '2016-04-04', 'Final', 120, 144),
 (446871, '2016-04-04 23:05:00', '2016-04-04', 'Final', 119, 135),
 (446878, '2016-04-04 23:10:00', '2016-04-04', 'Final', 141, 139),
 (446869, '2016-04-05 01:40:00', '2016-04-04', 'Final', 115, 109),
 (446889, '2016-04-05 02:05:00', '2016-04-04', 'Final', 112, 108),
 (446868, '2016-04-05 02:05:00', '2016-04-04', 'Final', 145, 133),
 (446896, '2016-04-05 20:15:00', '2016-04-05', 'Final', 121, 118),
 (446910, '2016-04-05 23:05:00', '2016-04-05', 'Final', 138, 1

In [ ]:
statement = """INSERT IGNORE INTO games 
               (gameid, game_end_datetime, game_date, status, home_id, away_id)
               VALUES (%s, %s, %s, %s, %s, %s)"""

In [ ]:
for i in range(len(tuples)):
    try:
        mycursor.execute(statement, tuples[i])
        mydb.commit()
    except:
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27